# DW_oa_Task04
## python自动化之PDF
## 20210620

### Python 操作 PDF

PDF 操作是本次自动化办公的最后一个知识点，初级的 PDF 自动化包括 PDF 文档的拆分、合并、提取等操作，更高级的还包括 WORD与PDF互转等

初级操作一般比较常用，也可以解决较多的办公内容，所以本节将会主要介绍 PDF 的初级操作，具体内容将会从以下几个小节展开：

1. 相关介绍
2. 批量拆分
3. 批量合并
4. 提取文字内容
5. 提取表格内容
6. 提取图片内容
7. 转换为PDF图片
8. 添加水印
9. 加密与解码

下面直接开始本节内容。

#### 1. 相关介绍

Python 操作 PDF 会用到两个库，分别是：PyPDF2 和 pdfplumber

其中 **PyPDF2** 可以更好的读取、写入、分割、合并PDF文件，而 **pdfplumber** 可以更好的读取 PDF 文件中内容和提取 PDF 中的表格

对应的官网分别是：

> PyPDF2：https://pythonhosted.org/PyPDF2/
>
> pdfplumber：https://github.com/jsvine/pdfplumber

由于这两个库都不是 Python 的标准库，所以在使用之前都需要单独安装

win+r 后输入 cmd 打开 command 窗口，依次输入如下命令进行安装：

In [1]:
!pip install pypdf2

In [2]:
!pip install --user pdfplumber

#### 2. 批量拆分

将一个完整的 PDF 拆分成几个小的 PDF，因为主要涉及到 PDF 整体的操作，所以本小节需要用到 PyPDF2 这个库

拆分的大概思路如下：

- 读取 PDF 的整体信息、总页数等
- 遍历每一页内容，以每个 step 为间隔将 PDF 存成每一个小的文件块
- 将小的文件块重新保存为新的 PDF 文件

需要注意的是，在拆分的过程中，可以手动设置间隔，例如：每5页保存成一个小的 PDF 文件

拆分的代码如下：

In [3]:
def split_pdf(filename, filepath, save_dirpath, step=5):
    """
    拆分PDF为多个小的PDF文件，
    @param filename:文件名
    @param filepath:文件路径
    @param save_dirpath:保存小的PDF的文件路径
    @param step: 每step间隔的页面生成一个文件，例如step=5，表示0-4页、5-9页...为一个文件
    @return:
    """
    if not os.path.exists(save_dirpath):
        os.mkdir(save_dirpath)   # 没有保存的文件路径,则新建
    pdf_reader = PdfFileReader(filepath)
    # 读取每一页的数据
    pages = pdf_reader.getNumPages()
    for page in range(0, pages, step):
        pdf_writer = PdfFileWriter()
        # 拆分pdf，每 step 页的拆分为一个文件
        for index in range(page, page+step):
            if index < pages:
                pdf_writer.addPage(pdf_reader.getPage(index))
        # 保存拆分后的小文件
        save_path = os.path.join(save_dirpath, filename+str(int(page/step)+1)+'.pdf')
        print(save_path)
        with open(save_path, "wb") as out:
            pdf_writer.write(out)

    print("文件已成功拆分，保存路径为："+save_dirpath)

In [4]:
def split_pdf1(filename, filepath, save_dirpath, step=5):
    """
    拆分PDF为多个小的PDF文件，
    @param filename:文件名
    @param filepath:文件路径
    @param save_dirpath:保存小的PDF的文件路径
    @param step: 每step间隔的页面生成一个文件，例如step=5，表示0-4页、5-9页...为一个文件
    @return:
    """
    if not os.path.exists(save_dirpath):
        os.mkdir(save_dirpath)   # 没有保存的文件路径,则新建
    pdf_reader = PdfFileReader(filepath)
    # 读取每一页的数据
    pages = pdf_reader.getNumPages()
    n = 0
    for page in range(0, pages):
        pdf_writer = PdfFileWriter()
        # 拆分pdf，每 step 页的拆分为一个文件
        pdf_writer.addPage(pdf_reader.getPage(page))
        save_path = os.path.join(save_dirpath, filename+str(int(page/step)+1)+'.pdf')
        out = open(save_path, "ab+")
        fy = n+step
        print(page)
        if fy > pages:
            fy = pages
        if page == fy-1:
            n = fy
            pdf_writer.write(out)
            # 保存拆分后的小文件
            print(save_path)
    print("文件已成功拆分，保存路径为："+save_dirpath)

In [5]:
import os
from PyPDF2 import PdfFileReader, PdfFileWriter
split_pdf('good','D:\\pythontest\\DW_oa\\tempfile.pdf','D:\\pythontest\\DW_oa\\temp',step=3)

D:\pythontest\DW_oa\temp\good1.pdf
D:\pythontest\DW_oa\temp\good2.pdf
D:\pythontest\DW_oa\temp\good3.pdf
文件已成功拆分，保存路径为：D:\pythontest\DW_oa\temp


In [6]:
split_pdf1('cool','D:\\pythontest\\DW_oa\\tempfile.pdf','D:\\pythontest\\DW_oa\\temp1',step=3)

0
1
2
D:\pythontest\DW_oa\temp1\cool1.pdf
3
4
5
D:\pythontest\DW_oa\temp1\cool2.pdf
6
7
D:\pythontest\DW_oa\temp1\cool3.pdf
文件已成功拆分，保存路径为：D:\pythontest\DW_oa\temp1


用自己的思维重写了一下,发现运行结果有问题,是思维逻辑的原因,理不出来,回头再来看看

**需要注意的是：**

如果你是第一次运行代码，在运行过程中，会直接报如下的错误

![](https://raw.githubusercontent.com/double-point/GraphBed/master/python_2_pdf/%E6%8B%86%E5%88%86%E6%8A%A5%E9%94%99.png)

如果是在 Pycharm 下，直接通过报错信息，点击 utils.py 文件，定位到第 238 行原文

原文中是这样的：

```python
 r = s.encode('latin-1')
 if len(s) < 2:
   		bc[s] = r
 return r
```

修改为：

```python
try:
    r = s.encode('latin-1')
    if len(s) < 2:
        bc[s] = r
    return r
except Exception as e:
    r = s.encode('utf-8')
    if len(s) < 2:
        bc[s] = r
    return r
```

如果你使用的是 **anaconda**，对应的文件路径应该为：anaconda\Lib\site-packages\PyPDF2\utils.py，进行同样的修改操作即可


#### 3. 批量合并

比起拆分来，合并的思路更加简单：

- 确定要合并的 **文件顺序**
- 循环追加到一个文件块中
- 保存成一个新的文件

对应的代码比较简单，基本不会出现问题：

In [7]:
def concat_pdf(filename, read_dirpath, save_filepath):
    """
    合并多个PDF文件
    @param filename:文件名
    @param read_dirpath:要合并的PDF目录
    @param save_filepath:合并后的PDF文件路径
    @return:
    """
    pdf_writer = PdfFileWriter()
    # 对文件名进行排序
    list_filename = os.listdir(read_dirpath)
    list_filename.sort(key=lambda x: int(x[:-4].replace(filename, "")))
    for filename in list_filename:
        print(filename)
        filepath = os.path.join(read_dirpath, filename)
        # 读取文件并获取文件的页数
        pdf_reader = PdfFileReader(filepath)
        pages = pdf_reader.getNumPages()
        # 逐页添加
        for page in range(pages):
            pdf_writer.addPage(pdf_reader.getPage(page))
    # 保存合并后的文件
    with open(save_filepath, "wb") as out:
        pdf_writer.write(out)
    print("文件已成功合并，保存路径为："+save_filepath)

In [8]:
concat_pdf('good', 'D:\\pythontest\\DW_oa\\temp', 'D:\\pythontest\\DW_oa\\good.pdf')

good1.pdf
good2.pdf
good3.pdf
文件已成功合并，保存路径为：D:\pythontest\DW_oa\good.pdf


#### 4. 提取文字内容

涉及到具体的 PDF 内容 操作，本小节需要用到 pdfplumber 这个库

在进行文字提取的时候，主要用到 extract_text 这个函数

具体代码如下：

In [9]:
import pdfplumber

In [10]:
def extract_text_info(filepath):
    """
    提取PDF中的文字
    @param filepath:文件路径
    @return:
    """
    with pdfplumber.open(filepath) as pdf:
        # 获取第2页数据
        page = pdf.pages[1]
        print(page.extract_text())

In [11]:
extract_text_info('D:\\pythontest\\DW_oa\\tempfile.pdf')

3、公司控股股东中国联合网络通信集团有限公司收到国资委出具的《关
于中国联合网络通信股份有限公司实施首期限制性股票激励计划的批复》(国
资考分[2017]1309 号)，原则同意中国联通实施限制性股票激励计划，以及限
制性股票激励计划的业绩考核目标。 
4、2018 年 1 月 16 日至 2018 年 1 月 29 日期间，公司通过内部网站或各
下属单位公告栏等途径，在公司内部公示了激励对象的姓名和职务。 
5、2018 年 2 月 9 日，公司召开第六届董事会第一次会议，审议通过了《关
于公司限制性股票激励计划、首期授予方案及相关办法草案修订稿和授予名
单的议案》。公司独立董事就本次限制性股票激励计划修订有利于公司的持续
发展及不存在损害公司与全体股东利益的情形发表独立意见。 
6、2018 年 2 月 9 日，公司召开第六届监事会第一次会议，审议通过了《关
于公司限制性股票激励计划、首期授予方案及相关办法草案修订稿的议案》
及《关于公司限制性股票激励计划首期授予方案激励对象审核意见的议案》。 
7、公司于 2018 年 2 月 12 日公告了《关于监事会对公司限制性股票激励
计划首期授予方案激励对象的审核意见及公示情况说明》。 
8、2018 年 2 月 28 日，公司 2018 年第二次临时股东大会审议并通过了《关
于公司限制性股票激励计划(草案修订稿)及其摘要的议案》《关于公司限制性
股票计划首期授予方案(草案修订稿)及其摘要的议案》及其他相关议案。公
司对限制性股票激励计划内幕信息知情人买卖公司股票情况进行了自查，未
发现相关内幕信息知情人存在利用内幕信息买卖公司股票的行为。 
9、2018 年 3 月 15 日，公司第六届董事会第二次会议审议通过了《关于
公司限制性股票激励计划首期授予方案实施授予相关事项的议案》，公司独立
董事对该议案发表了独立意见。 
10、2018 年 3 月 15 日，公司第六届监事会第二次会议审议通过了《关于
审议董事会<关于公司限制性股票激励计划首期授予方案实施授予相关事项
的议案>的议案》，并对相关事项发表了核查意见。 
  2 


>  拓展一下：此处可以结合前面 word 小节，将内容写入 word 文件中

In [12]:
pdf =  pdfplumber.open('D:\\pythontest\\DW_oa\\tempfile.pdf')
page = pdf.pages[1]
text = page.extract_text()   # 读取第二页内容

In [13]:
from docx import Document
doc_1 = Document()
paragraph_1 = doc_1.add_paragraph(text)
doc_1.save('doc_pdf.docx')

简单写入,没有排版及设置,中间过程思维错误,以为输出结果是字符串,将函数直接赋值给变量,发现保存文档一直为空,后来才找出问题所在

#### 5. 提取表格内容

同样的，本节是对具体内容的操作，所以也需要用到 pdfplumber 这个库

和提取文字十分类似的是，提取表格内容只是将 extract_text 函数换成了 extract_table 函数

对应的代码如下：

In [14]:
def extract_table_info(filepath):
    """
    提取PDF中的图表数据
    @param filepath:
    @return:
    """
    with pdfplumber.open(filepath) as pdf:
        # 获取第18页数据
        page = pdf.pages[5]
        # 如果一页有一个表格，设置表格的第一行为表头，其余为数据
        table_info = page.extract_table()
        df_table = pd.DataFrame(table_info[1:], columns=table_info[0])
        df_table.to_csv('dmeo.csv', index=False, encoding='gbk')

上面代码可以获取到第 18 页的第一个表格内容，并且将其保存为 csv 文件存在本地

> 但是，如果说第 18 页有多个表格内容呢？

因为读取的表格会被存成二维数组，而多个二维数组就组成一个三维数组

遍历这个三位数组，就可以得到该页的每一个表格数据，对应的将 extract_table 函数 改成 extract_tables 即可

具体代码如下：

In [15]:
# 如果一页有多个表格，对应的数据是一个三维数组
tables_info = page.extract_tables()
for index in range(len(tables_info)):
    # 设置表格的第一行为表头，其余为数据
    df_table = pd.DataFrame(tables_info[index][1:], columns=tables_info[index][0])
    print(df_table)
    # df_table.to_csv('dmeo.csv', index=False, encoding='gbk')

In [16]:
import pandas as pd
extract_table_info('D:\\pythontest\\DW_oa\\tempfile.pdf')

In [17]:
df = pd.read_csv('dmeo.csv', encoding='gbk')
df

,类别,变动前,本次变动,变动后
0,有限售条件股份,"9,515,684,692","-3,742,200","9,511,942,492"
1,无限售条件股份2,"21,499,910,795",0,"21,499,910,795"
2,股份总数,"31,015,595,487","-3,742,200","31,011,853,287"


#### 6. 提取图片内容

提取 PDF 中的图片和将 PDF 转存为图片是不一样的（下一小节），需要区分开。

提取图片：顾名思义，就是将内容中的图片都提取出来；转存为图片：则是将每一页的 PDF 内容存成一页一页的图片，下一小节会详细说明

转存为图片中，需要用到一个模块叫 fitz，fitz 的最新版 1.18.13，非最新版的在部分函数名称上存在差异，代码中会标记出来

使用 fitz 需要先安装 PyMuPDF 模块，安装方式如下：

In [18]:
!pip install PyMuPDF

提取图片的整体逻辑如下：

- 使用 fitz 打开文档，获取文档详细数据
- 遍历每一个元素，通过正则找到图片的索引位置
- 使用 Pixmap 将索引对应的元素生成图片
- 通过 size 函数过滤较小的图片

实现的具体代码如下：

In [19]:
import fitz
import re

In [20]:
pic_dirpath = 'D:\\pythontest\\DW_oa\\pic'
filepath = 'D:\\pythontest\\DW_oa\\zoom.pdf'

if not os.path.exists(pic_dirpath):
    os.makedirs('D:\\pythontest\\DW_oa\\pic')
# 使用正则表达式来查找图片
check_XObject = r"/Type(?= */XObject)"
check_Image = r"/Subtype(?= */Image)"
img_count = 0

"""1. 打开pdf，打印相关信息"""
pdf_info = fitz.open(filepath)
# 1.16.8版本用法 xref_len = doc._getXrefLength()
# 最新版本写法
xref_len = pdf_info.xref_length()
# 打印PDF的信息
print("文件名：{}, 页数: {}, 对象: {}".format(filepath, len(pdf_info), xref_len-1))

"""2. 遍历PDF中的对象，遇到是图像才进行下一步，不然就continue"""
for index in range(1, xref_len):
    # 1.16.8版本用法 text = doc._getXrefString(index)
    # 最新版本
    text = pdf_info.xref_object(index)
    
    is_XObject = re.search(check_XObject, text)
    is_Image = re.search(check_Image, text)
    # 如果不是对象也不是图片，则不操作
    if is_XObject or is_Image:
        img_count += 1
        # 根据索引生成图像
        pix = fitz.Pixmap(pdf_info, index)
        pic_filepath = os.path.join(pic_dirpath, 'img_' + str(img_count) + '.png')
        """pix.size 可以反映像素多少，简单的色素块该值较低，可以通过设置一个阈值过滤。以阈值 10000 为例过滤"""
        # if pix.size < 10000:
        #     continue
        
        """三、 将图像存为png格式"""
        if pix.n >= 5:
            # 先转换CMYK
            pix = fitz.Pixmap(fitz.csRGB, pix)
        # 存为PNG
        pix.writePNG(pic_filepath)

文件名：D:\pythontest\DW_oa\zoom.pdf, 页数: 1, 对象: 12


#### 7. 转换为图片

转换为照片比较简单，就是将一页页的 PDF 转换为一张张的图片。大致过程如下：


##### 7.1 安装 pdf2image

首先需要安装对应的库，最新的 pdf2image 库版本应该是 1.14.0

它的 github地址 为：https://github.com/Belval/pdf2image ，感兴趣的可以自行了解

安装方式如下：

In [21]:
!pip install pdf2image

##### 7.2 安装组件

对于不同的平台，需要安装相应的组件，这里以 windows 平台和 mac 平台为例：

**Windows 平台**

对于 windows 用户需要安装 poppler for Windows，安装链接是：http://blog.alivate.com.au/poppler-windows/

另外，还需要添加环境变量， 将 bin 文件夹的路径添加到环境变量 PATH 中 

> 注意这里配置之后需要重启一下电脑才会生效，不然会报如下错误：

**Mac**

对于 mac 用户，需要安装 poppler for Mac，具体可以参考这个链接：http://macappstore.org/poppler/

<br>

详细代码如下：

In [22]:
import pdf2image
pic_dirpath = 'D:\\pythontest\\DW_oa\\pic'
filepath = 'D:\\pythontest\\DW_oa\\tempfile.pdf'

if not os.path.exists(pic_dirpath):
    os.makedirs(pic_dirpath)

images = pdf2image.convert_from_bytes(open(filepath, 'rb').read())
# images = convert_from_path(filepath, dpi=200)
for image in images:
    # 保存图片
    pic_filepath = os.path.join(pic_dirpath, 'img_'+str(images.index(image))+'.png')
    image.save(pic_filepath, 'PNG')

这个组件怎么处理,还没搞定,错误提示百度没找到,太费时间,先做下一步

#### 8. 添加水印

PDF 中添加水印，首先需要一个水印PDF文件，然后依次通过 mergePage 操作将每一页的 PDF 文件合并到水印文件上，据此，每一页的 PDF 文件将是一个带有水印的 PDF 文件

最后，将每一页的水印 PDF 合并成一个 PDF 文件即可

**生成水印**

生成水印的方式比较多，例如在图片添加水印，然后将图片插入到 word 中，最后将 word 保存成 PDF 文件即可

生成一张 A4 纸大小的空白图片，参考这篇文章：[Python 批量加水印！轻松搞定！](https://mp.weixin.qq.com/s/_oJA6lbsdMlRRsBf6DPxsg) 给图片添加水印，最终的水印背景图片是这样的：

![](https://raw.githubusercontent.com/double-point/GraphBed/master/python_2_pdf/%E7%A9%BA%E7%99%BD%E7%85%A7%E7%89%87.png)

然后将图片插入到 word 中并最终生成一个水印 PDF 文档

PDF 文档添加水印的主要代码如下：

In [23]:
from copy import copy
filepath = 'D:\\pythontest\\DW_oa\\tempfile.pdf'
watermark_filepath = 'D:\\pythontest\\DW_oa\\zoom.pdf'
save_filepath = 'D:\\pythontest\\DW_oa\\markfile.pdf'

watermark = PdfFileReader(watermark_filepath)
watermark_page = watermark.getPage(0)

pdf_reader = PdfFileReader(filepath)
pdf_writer = PdfFileWriter()

for page_index in range(pdf_reader.getNumPages()):
    current_page = pdf_reader.getPage(page_index)
    # 封面页不添加水印
    if page_index == 0:
        new_page = current_page
    else:
        new_page = copy(watermark_page)
        new_page.mergePage(current_page)
    pdf_writer.addPage(new_page)
# 保存水印后的文件
with open(save_filepath, "wb") as out:
    pdf_writer.write(out)

#### 9. 文档加密与解密

你可能在打开部分 PDF 文件的时候，会弹出下面这个界面：

![](https://raw.githubusercontent.com/double-point/GraphBed/master/python_2_pdf/PDF%E5%B7%B2%E5%8A%A0%E5%AF%86.png)

这种就是 PDF 文件被加密了，在打开的时候需要相应的密码才行

本节所提到的也只是基于 PDF 文档的加密解密，而不是所谓的 PDF 密码破解。

在对 PDF 文件加密需要使用 encrypt 函数，对应的加密代码也比较简单：

In [24]:
passwd = '123456'
save_filepath = 'D:\\pythontest\\DW_oa\\passwd.pdf'
pdf_reader = PdfFileReader(filepath)
pdf_writer = PdfFileWriter()

for page_index in range(pdf_reader.getNumPages()):
    pdf_writer.addPage(pdf_reader.getPage(page_index))

# 添加密码
pdf_writer.encrypt(passwd)
with open(save_filepath, "wb") as out:
    pdf_writer.write(out)

代码执行成功后再次打开 PDF 文件则需要输入密码才行

根据这个思路，破解 PDF 也可以通过暴力求解实现，例如：通过本地密码本一个个去尝试，或者根据数字+字母的密码形式循环尝试，最终成功打开的密码就是破解密码

> 上述破解方法耗时耗力，不建议尝试

另外，针对已经加密的 PDF 文件，也可以使用 decrypt 函数进行解密操作

解密代码如下：

In [25]:
filepath = 'D:\\pythontest\\DW_oa\\passwd.pdf'
save_filepath = 'D:\\pythontest\\DW_oa\\unpasswd.pdf'

pdf_reader = PdfFileReader(filepath)
# PDF文档解密
pdf_reader.decrypt(passwd)

pdf_writer = PdfFileWriter()
for page_index in range(pdf_reader.getNumPages()):
    pdf_writer.addPage(pdf_reader.getPage(page_index))

with open(save_filepath, "wb") as out:
    pdf_writer.write(out)

上面还有两个问题没有解决,今天没时间了,再查查原因吧